<a href="https://colab.research.google.com/github/viniciuswv/Hybrid-MPSI-CoCoSo/blob/main/Hybrid_MPSI_CoCoSo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#----------------------------------------------------------------------------------------------------------
# ESCOLA DE GUERRA NAVAL - MARINHA DO BRASIL
# Programa de Pós-graduação em Estudos Marítimos (PPGEM) - Doutorado Profissional
# Desenvolvedor: Vinicius Wittig Vianna, MSc.
# Orientador: CMG Adriano Lauro, DSc.

# Hybrid MPSI-CoCoSo (ver. 1.0.0 - 07/10/2024)

# /Notas do autor/
# O programa desenvolvido abaixo é um modelo híbrido de análise multicritério composto em dois estágios.
# Inicialmente, o modelo calcula os pesos de cada critério e, após isso, calcula a ordenação das alternativas.
# Para maiores detalhes, gentileza verificar o arquivo Readme.
#----------------------------------------------------------------------------------------------------------


import pandas as pd
import numpy as np
from google.colab import files

# Função para normalizar a matriz de decisão de acordo com o tipo de critério (MPSI)
def normalize_matrix_mpsi(matrix, criteria_types):
    normalized_matrix = np.zeros(matrix.shape)
    for j, criterion in enumerate(criteria_types):
        if criterion.lower() == 'max':
            normalized_matrix[:, j] = matrix[:, j] / np.max(matrix[:, j])
        elif criterion.lower() == 'min':
            normalized_matrix[:, j] = np.min(matrix[:, j]) / matrix[:, j]
    return normalized_matrix

# Função para normalizar a matriz de decisão de acordo com o tipo de critério (CoCoSo)
def normalize_matrix_cocoso(matrix, criteria_types):
    normalized_matrix = np.zeros(matrix.shape)
    for j, criterion in enumerate(criteria_types):
        min_val = np.min(matrix[:, j])
        max_val = np.max(matrix[:, j])
        if max_val == min_val:
            normalized_matrix[:, j] = 1  # Evita divisão por zero
        else:
            if criterion.lower() == 'max':
                normalized_matrix[:, j] = (matrix[:, j] - min_val) / (max_val - min_val)
            elif criterion.lower() == 'min':
                normalized_matrix[:, j] = (max_val - matrix[:, j]) / (max_val - min_val)
    return normalized_matrix

# Método MPSI para calcular pesos dos critérios
def mpsi_weights(matrix, criteria_types):
    normalized_matrix = normalize_matrix_mpsi(matrix, criteria_types)
    vj = np.mean(normalized_matrix, axis=0)
    pj = np.sum((normalized_matrix - vj) ** 2, axis=0)
    weights = pj / np.sum(pj)
    return weights

# Método CoCoSo para ordenação das alternativas
def cocoso_ranking(matrix, weights, criteria_types, λ):
    normalized_matrix = normalize_matrix_cocoso(matrix, criteria_types)
    Si = np.sum(weights * normalized_matrix, axis=1)
    Pi = np.sum(normalized_matrix ** weights, axis=1)
    min_Si = np.min(Si) if np.min(Si) != 0 else 1e-10
    min_Pi = np.min(Pi) if np.min(Pi) != 0 else 1e-10
    kia = (Pi + Si) / np.sum(Pi + Si)
    kib = Si / min_Si + Pi / min_Pi
    kic = (λ * Si + (1 - λ) * Pi) / (λ * np.max(Si) + (1 - λ) * np.max(Pi))
    ki = (kia * kib * kic) ** (1 / 3) + (1 / 3) * (kia + kib + kic)
    return ki

# Função principal para carregar a planilha e executar os métodos MPSI e CoCoSo
def main():
    print("Por favor, carregue o arquivo Excel com a matriz de decisão (primeira linha deve conter os títulos).")
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]

    df = pd.read_excel(file_name, sheet_name=0)

    alternatives = df.iloc[:, 0].values
    criteria_matrix = df.iloc[:, 1:].values
    criteria_matrix = criteria_matrix.astype(float)
    criteria_names = df.columns[1:]

    print(f"Critérios encontrados: {', '.join(criteria_names)}")
    criteria_types = input(f"Informe se cada critério é de maximização ou minimização (ex: max, min, ...): ").split(", ")

    # Etapa 1: Calcular pesos dos critérios
    print("Calculando pesos dos critérios...")
    weights = mpsi_weights(criteria_matrix, criteria_types)
    print(f"Pesos calculados para cada critério:")
    for crit, weight in zip(criteria_names, weights):
        print(f"Critério: {crit}, Peso: {weight:.4f}")

    # Solicitar valor de λ ao usuário
    while True:
        try:
            λ = float(input("Informe um valor para λ (entre 0 e 1): "))
            if 0 <= λ <= 1:
                break
            else:
                print("Por favor, insira um valor entre 0 e 1.")
        except ValueError:
            print("Valor inválido. Por favor, insira um número decimal entre 0 e 1.")

    # Etapa 2: Ordenação das alternativas
    print("Calculando ordenação das alternativas...")
    ranking_values = cocoso_ranking(criteria_matrix, weights, criteria_types, λ)

    # Gerar dataframe com os resultados finais
    results = pd.DataFrame({'Alternativa': alternatives, 'Valor CoCoSo': ranking_values})

    # Ordenar as alternativas pela melhor (maior valor Ki) para pior
    results = results.sort_values(by='Valor CoCoSo', ascending=False)

    # Definir as posições com base no valor de Ki (melhor para pior)
    results['Posição'] = range(1, len(results) + 1)
    results['Posição'] = results['Posição'].apply(lambda x: f"{x}º")

    # Solicitar ao usuário como deseja visualizar a lista (crescente ou decrescente)
    order_preference = input("Deseja visualizar a lista na ordem crescente (1º, 2º, 3º, ...) ou decrescente (3º, 2º, 1º, ...)? Digite 'crescente' ou 'decrescente': ").lower()

    # Ajustar a lista de exibição conforme a escolha do usuário
    if order_preference == 'crescente':
        display_results = results
    elif order_preference == 'decrescente':
        display_results = results.iloc[::-1]  # Inverter a lista
    else:
        print("Opção inválida! Exibindo por padrão na ordem crescente.")
        display_results = results

    # Exibir pesos calculados
    print("\n--- Pesos calculados para cada critério ---")
    for crit, weight in zip(criteria_names, weights):
        print(f"{crit}: {weight:.4f}")

    # Exibir ordenação final conforme a preferência do usuário
    print("\n--- Ordenação final das alternativas ---")
    for pos, alt, val in zip(display_results['Posição'], display_results['Alternativa'], display_results['Valor CoCoSo']):
        print(f"{pos} lugar: {alt}, Agregação final (Ki): {val:.4f}")

# Executar o programa principal
if __name__ == "__main__":
    main()

Por favor, carregue o arquivo Excel com a matriz de decisão (primeira linha deve conter os títulos).


Saving Matriz_Decisao_Fornecedores.xlsx to Matriz_Decisao_Fornecedores (13).xlsx
Critérios encontrados: Custo (C1), Qualidade (C2), Tempo de Resposta (C3), Reputação (C4)
Informe se cada critério é de maximização ou minimização (ex: max, min, ...): max, min, max, min
Calculando pesos dos critérios com MPSI...
Pesos calculados para cada critério (MPSI):
Critério: Custo (C1), Peso: 0.2265
Critério: Qualidade (C2), Peso: 0.2265
Critério: Tempo de Resposta (C3), Peso: 0.3205
Critério: Reputação (C4), Peso: 0.2265
Informe um valor para λ (entre 0 e 1): 0.1
Calculando ordenação das alternativas com CoCoSo...
Deseja visualizar a lista na ordem crescente (1º, 2º, 3º, ...) ou decrescente (3º, 2º, 1º, ...)? Digite 'crescente' ou 'decrescente': crescente

--- Pesos calculados para cada critério (MPSI) ---
Custo (C1): 0.2265
Qualidade (C2): 0.2265
Tempo de Resposta (C3): 0.3205
Reputação (C4): 0.2265

--- Ordenação final das alternativas (CoCoSo) ---
1º lugar: Fornecedor B, Agregação final (Ki): 2